In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mode"
fs_method, fs_ratio = "fcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9150 | AUPR: 0.6526 | Acc: 95.38%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9191 | AUPR: 0.6625 | Acc: 95.35%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9215 | AUPR: 0.6690 | Acc: 95.29%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9142 | AUPR: 0.6538 | Acc: 95.29%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9191 | AUPR: 0.6647 | Acc: 95.32%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9208 | AUPR: 0.6708 | Acc: 95.38%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9215 | AUPR: 0.6690 | Acc: 95.29%
Internal test: AUC: 0.9255 | AUPR: 0.7008 | Acc: 95.35%
External test: AUC: 0.9235 | AUPR: 0.6686 | Acc: 95.19%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9151 | AUPR: 0.6206 | Acc: 94.91%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9155 | AUPR: 0.6148 | Acc: 94.93%
model(kernel=linear, C=1)
Valid:         AUC: 0.9152 | AUPR: 0.6111 | Acc: 94.94%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9155 | AUPR: 0.6148 | Acc: 94.93%
Internal test: AUC: 0.9212 | AUPR: 0.6667 | Acc: 94.84%
External test: AUC: 0.9164 | AUPR: 0.6001 | Acc: 94.50%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7486 | AUPR: 0.4206 | Acc: 94.74%
model(n_neighbors=5)
Valid:         AUC: 0.7796 | AUPR: 0.4744 | Acc: 94.96%
model(n_neighbors=7)
Valid:         AUC: 0.7938 | AUPR: 0.4950 | Acc: 95.04%
best model(n_neighbors=7)
Valid:         AUC: 0.7938 | AUPR: 0.4950 | Acc: 95.04%
Internal test: AUC: 0.7909 | AUPR: 0.5141 | Acc: 94.91%
External test: AUC: 0.8011 | AUPR: 0.5029 | Acc: 94.87%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8719 | AUPR: 0.3981 | Acc: 86.03%
best model()
Valid:         AUC: 0.8719 | AUPR: 0.3981 | Acc: 86.03%
Internal test: AUC: 0.8846 | AUPR: 0.4294 | Acc: 85.79%
External test: AUC: 0.8727 | AUPR: 0.3811 | Acc: 84.43%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6487 | Acc: 95.20%
model(C=10.0)
Valid:         AUC: 0.9162 | AUPR: 0.6478 | Acc: 95.21%
model(C=100.0)
Valid:         AUC: 0.9157 | AUPR: 0.6476 | Acc: 95.19%
best model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6487 | Acc: 95.20%
Internal test: AUC: 0.9222 | AUPR: 0.6847 | Acc: 95.18%
External test: AUC: 0.9161 | AUPR: 0.6303 | Acc: 94.81%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8619 | AUPR: 0.5267 | Acc: 80.90%
model(criterion=log_loss)
Valid:         AUC: 0.8665 | AUPR: 0.5683 | Acc: 78.17%
model(criterion=entropy)
Valid:         AUC: 0.8665 | AUPR: 0.5683 | Acc: 78.17%
best model(criterion=log_loss)
Valid:         AUC: 0.8665 | AUPR: 0.5683 | Acc: 78.17%
Internal test: AUC: 0.8603 | AUPR: 0.5969 | Acc: 79.66%
External test: AUC: 0.8636 | AUPR: 0.5670 | Acc: 79.90%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9082 | AUPR: 0.6487 | Acc: 88.76%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9092 | AUPR: 0.6478 | Acc: 88.78%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9099 | AUPR: 0.6474 | Acc: 88.64%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9081 | AUPR: 0.6531 | Acc: 89.41%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9095 | AUPR: 0.6532 | Acc: 89.26%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9098 | AUPR: 0.6497 | Acc: 88.98%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9081 | AUPR: 0.6531 | Acc: 89.41%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9095 | AUPR: 0.6532 | Acc: 89.26%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9098 | AUPR: 0.6497 | Acc: 88.98%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9099 | AUPR: 0.6474 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9167 | AUPR: 0.6514 | Acc: 95.18%
model(criterion=squared_error)
Valid:         AUC: 0.9166 | AUPR: 0.6510 | Acc: 95.16%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9167 | AUPR: 0.6514 | Acc: 95.18%
Internal test: AUC: 0.9238 | AUPR: 0.6831 | Acc: 95.01%
External test: AUC: 0.9200 | AUPR: 0.6636 | Acc: 95.05%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9135 | AUPR: 0.6315 | Acc: 95.13%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9143 | AUPR: 0.6340 | Acc: 95.19%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6343 | Acc: 95.15%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6343 | Acc: 95.15%
Internal test: AUC: 0.9185 | AUPR: 0.6805 | Acc: 95.30%
External test: AUC: 0.9153 | AUPR: 0.6218 | Acc: 94.58%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)